In [62]:
%pip install --upgrade tensorflow
%pip install pandas nltk sentence_transformers transformers
import pandas as pd
from bs4 import BeautifulSoup
import re

from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt', download_dir=nltk.data.path[0])

import faiss
import numpy as np
import json

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to C:\Users\shepa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

# Load your CSV
fridge_data = r'data\refridgerator_scrape.csv'
dishwasher_data = r'data\dishwasher_scrape.csv'
df = pd.read_csv(fridge_data)
df1 = pd.read_csv(dishwasher_data)
combined_df = pd.concat([df, df1], ignore_index=True, axis=0)
# Function to clean HTML and remove escape characters
def clean_html(raw_html):
    if pd.isna(raw_html):
        return ''
    
    # Strip HTML tags
    text = BeautifulSoup(raw_html, 'html.parser').get_text(separator=' ', strip=True)
    
    # Remove \r, \n, \t and extra spaces
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply to all columns
df_cleaned = combined_df.applymap(clean_html)
df_cleaned.to_csv("test.csv", index=False)
# Show the cleaned DataFrame
df_cleaned.head(1)




C:\Users\shepa\AppData\Local\Temp\ipykernel_28924\2310578413.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = combined_df.applymap(clean_html)
C:\Users\shepa\AppData\Local\Temp\ipykernel_28924\2310578413.py:13: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import w

,nf__links href,BODY,container,repair-story,section-title,repair-story (2),repair-story (3),container (2),d-flex,d-flex (2),...,d-flex (4),d-flex (5),no-underline href,schematic-thump__img src,no-underline href (2),schematic-thump__img src (2),no-underline href (3),schematic-thump__img src (3),no-underline href (4),schematic-thump__img src (4)
0,https://www.partselect.com/Refrigerator-Trays-...,Skip to main content 1-866-319-8402 Monday to ...,Refrigerator Trays and Shelves Find Your Part ...,In Stock Item is in stock and will ship today ...,In Stock Item is in stock and will ship today ...,In Stock Item is in stock and will ship today ...,Feedback How would you rate your experience? H...,Refrigerator Trays and Shelves Find Your Part ...,,,...,,,,,,,,,,


In [ ]:
repair_stories = pd.read_csv(r'\data\repair_stories.csv')
repair_stories.head()

parts = pd.read_csv(r'\data\part_info_final.csv')

In [65]:
def combine_columns(row):
    # Iterate through all columns and concatenate the text values
    return ' | '.join([f"{col}: {row[col]}" for col in row.index if isinstance(row[col], str) and row[col].strip() != ""])

df_cleaned['combined_text'] = df_cleaned.apply(combine_columns, axis=1)
parts['combined_text'] = parts.apply(combine_columns, axis=1)
df_cleaned = df_cleaned[['combined_text']]
parts_cleaned = parts[['combined_text']]

repair_stories.rename(columns={'repair-story': 'combined_text'}, inplace=True)

full_text = pd.concat((df_cleaned, parts_cleaned, repair_stories), axis=0)
full_text.head()
full_text = full_text.drop(columns='Unnamed: 0')

In [66]:
full_text['combined_text'] = full_text['combined_text'].replace({r'[\n\t⭐]': ' '}, regex=True)
full_text['combined_text'] = full_text['combined_text'].replace({'★★★★★': ''}, regex=False)

full_text.head()

,combined_text
0,nf__links href: https://www.partselect.com/Ref...
1,nf__links href: https://www.partselect.com/Ref...
2,nf__links href: https://www.partselect.com/Ref...
3,nf__links href: https://www.partselect.com/Ref...
4,nf__links href: https://www.partselect.com/Ref...


In [67]:

model = SentenceTransformer('BAAI/bge-small-en')

# Load the BERT tokenizer (you can change this to a different BERT variant)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Download NLTK punkt tokenizer
# nltk.download('punkt')

# Function to split the text into chunks
def split_text_into_chunks(text, max_sentences_per_chunk=3):
    # Split text into sentences (naive split based on period)
    sentences = [s.strip() for s in text.split('.') if s.strip()]
    
    # Create chunks where each chunk has a maximum of `max_sentences_per_chunk` sentences
    chunks = ['. '.join(sentences[i:i + max_sentences_per_chunk]) + '.' for i in range(0, len(sentences), max_sentences_per_chunk)]
    
    return chunks

# Create lists for chunks and metadata
processed_chunks = []


chunk_set = set()
for idx, row in full_text.iterrows():
    combined_text = row['combined_text']
    
    # Split into 2–3 chunks
    chunks = split_text_into_chunks(combined_text)
    chunk_set.update(chunks)

    
for chunk_id, chunk in enumerate(chunk_set):
    tokens = tokenizer.tokenize(chunk)
    processed_chunks.append({
        "chunk_id": chunk_id,
        "original_text": chunk,
        "tokenized_text": tokens
    })    

In [68]:
import faiss
# index = faiss.read_index(r'data/embedded_chunks.faiss')


In [69]:
# Step 1: Prepare original text chunks for embedding
all_chunks = [entry["original_text"] for entry in processed_chunks]

# Step 2: Embed chunks
embeddings = model.encode(all_chunks, show_progress_bar=True)

# Step 3: Create FAISS index
embedding_matrix = np.vstack(embeddings).astype("float32")
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 = cosine similarity if embeddings are normalized
index.add(embedding_matrix)

# Step 4: Save FAISS index
faiss.write_index(index, "embedded_chunks.faiss")

# Step 5: Save metadata (original chunk, row info, etc.)


Batches: 100%|██████████| 402/402 [05:05<00:00,  1.31it/s]


In [70]:
with open("embedded_chunks_metadata.json", "w", encoding="utf-8") as f:
    json.dump(processed_chunks, f, indent=2, ensure_ascii=False)

In [71]:
print(len(all_chunks))

12856


In [72]:
print(index.ntotal)

12856
